In [ ]:
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
from search.collect_rss import collect
from collections import defaultdict

term = "harry%20potter"
# resp = urllib.request.urlopen("https://api.wattpad.com:443/v4/stories")
# soup = BeautifulSoup(response.data, 'html.parser')

site= "https://www.wattpad.com/search/" + term
hdr = {'User-Agent': 'Mozilla/5.0'}
req = Request(site,headers=hdr)
page = urlopen(req)
soup = BeautifulSoup(page, "lxml")


def find_fanfiction(top = 1):
    final = defaultdict(str)
    for link in soup.find_all('a', 'on-result')[:top]: #if too slow just take 1st or second link
        url = "https://www.wattpad.com" + link["href"] + "/parts"
        req = Request(url,headers=hdr)
        page = urlopen(req)
        soup2 = BeautifulSoup(page, "lxml")
        #ul class="table-of-contents         
        for link2 in soup2.find_all('ul', 'table-of-contents'):
            z = link2.find_all('a', href=True)
            listing = [ele["href"] for ele in z]
            d = defaultdict(BeautifulSoup)
            for x in listing:
                #print(url)
                list2 = BeautifulSoup(urlopen(Request("https://www.wattpad.com" + x, headers = hdr)), "lxml").find_all('p')
                
                paragraph = [str(j)[str(j).index('>')+1:str(j).rfind('<')] for j in list2 if "data-p-id" in str(j)]
                #paragraph = "".join(paragraph)
                final[term] += "\n".join(paragraph)
                print(final)
        
                #print("DONE boyz" + str(len(paragraph)))
                #print(paragraph)
#                     string = "" + str(j)
#                     if "data-p-id" in string:
#                         print(string[string.index('>')+1:string.rfind('<')])
#                         paragraph.append(string[string.index('>')+1:string.rfind('<')])
                #print(paragraph)
    print(len(final[term]))
find_fanfiction()
            #print(para)
            #print(len(list2))
            #print(list2)
#             paragraph = [x for x in list2 if "data-p-id" in x]#list comprehension
#             print(paragraph, "paragraph")
        #print(d)
        #print(listing, "listing ")
        #print(url)
        
#     l = link.find_all('a',  href=True)[0]["href"]
#     print(l)
#     p = collect(l, l[l.rfind("/")+1:]+".pickle")

In [ ]:
".".join([str(["a","b"])])

In [ ]:
def generate_text(fanfiction_dict):
    for key, value in fanfiction_dict:
        lm = self.train_lm(value, n=10)
        text = self.generate_text(lm, 10, n_letters=200)

In [ ]:
import re

your_string = '\xa0"I\'m sure it\'s nothing bad, hun. \xa0They probably want to tell us we\'ve got the next Einstein on our hands!" \xa0'
str(print(your_string))

In [22]:
import sys
sys.path.insert(0, 'C:/Users/User/Desktop/beaver/NumberWon/numberwon/alexa_skills/search')
print("done")
from entityDatabase import entityDatabase

done


In [ ]:
entityDatabase.

from collections import Counter, defaultdict
import pickle
import string
import nltk
import re
from nltk.tokenize import word_tokenize
from search.searchEngine import MySearchEngine

class entityDatabase:
    def __init__(self):
        self.ent_dict = defaultdict(list)
        self.ent_dict2 = defaultdict(list)
        self.engine = MySearchEngine()

    def get_by_id(self, id):
        return self.ent_dict[id]

    def entize(self, pickle, dictionary):
        #creates a dictionary where link = key, and value = list of entities
        for key, val in pickle.items():
            tokens = nltk.word_tokenize(val)
            pos = nltk.pos_tag(tokens)
            named_entities = nltk.ne_chunk(pos, binary = True)
            for i in range(0, len(named_entities)):
                ents = named_entities.pop()
                if getattr(ents, 'label', None) != None and ents.label() == "NE":
                    z = list(zip(*[ne for ne in ents]))[0]
                    z = " ".join(z)
                    dictionary[key].append(z)
        return dictionary

    def entize2(self, pickle, dictionary):
        counter = 0
        for key, val in pickle.items():
            tokens = nltk.word_tokenize(val)
            pos = nltk.pos_tag(tokens)
            named_entities = nltk.ne_chunk(pos, binary = True)
            for i in range(0, len(named_entities)):
                ents = named_entities.pop()
                if getattr(ents, 'label', None) != None and ents.label() == "NE":
                    z = list(zip(*[ne for ne in ents]))[0]
                    z = (" ".join(z), counter)
                    dictionary[key].append(z)
                counter += len(ents)
        return dictionary

    def searchNentity(self, qword):
        if self.engine.query(qword) != []:
            topdoc = self.engine.query(qword)[0][0]
            return self.top_entity_pos(qword,self.engine.raw_text[topdoc]) #Megan's method
        return None

    def docsearch(self, qword):
        if self.engine.query(qword) != []:
            topdoc = self.engine.query(qword)[0][0]
            raw = self.engine.raw_text[topdoc] #whole doc
            return re.match(r'(?:[^.:;]+[.:;]){1}', raw).group().replace('\n\nFILE PHOTO', "") #first sentence
        return None

    def get_title_and_first_sentence(self, qword):
        return self.engine.whats_new(qword)

    def top_entity_pos(self, item, most_c=10):
        #search for item.
            #for i in feed. if i == feed:
        #create a list of words that are close to word in proximity
        #score based on proximity to word.
        #documents is already a list
        word_freq = Counter()
        for i in self.ent_dict2:
            #print(self.ent_dict2[i])
            for x in self.ent_dict2[i]:
                if x[0] == item:
                    for z in self.ent_dict2[i]:
                        if x[0] != z[0]:
                            #print((abs(x[1]-z[1])))
                            word_freq[z[0]] += 1/(abs((x[1]-z[1])))

        return word_freq.most_common(most_c)

    def top_entity_dict(self, item, most_c=10):
        #documents is already a list
        #turn each list into a counter, add all counters together.
        mega_counter = Counter()
        for i in self.ent_dict:
            #get list of counters etc
            if item in self.ent_dict[i]:
                c = Counter(self.ent_dict[i])
                del c[item]
                mega_counter += c
        return mega_counter.most_common(most_c)

    def add_File_Database(self, pickle_path):
        p = pickle.load(open(pickle_path, "rb"))
        self.engine.upload_vd(pickle_path)
        # pickle_path example: "C:\\Users\\User\\Desktop\\beaver\\NumberWon\\numberwon\\entity\\test.pickle"
        self.ent_dict = self.entize(p, self.ent_dict)
        self.ent_dict2 = self.entize2(p, self.ent_dict2)

    def add_Folder_Database(self, path_pickle_folder):
        #have paths in the form '/path/to/dir'
        import os
        for f in os.listdir(path_pickle_folder):
            if f.endswith(".pickle"):
                p = pickle.load(open(path_pickle_folder + "/" + f, "rb"))
                self.engine.upload_vd(path_pickle_folder + "/" + f)
                self.ent_dict = self.entize(p, self.ent_dict)
                self.ent_dict2 = self.entize2(p, self.ent_dict2)
                print(f, type(f))

In [23]:
edatb = entityDatabase()
#edatb.ent_dict
edatb.add_Folder_Database('C:/Users/User/Desktop/beaver/NumberWon/numberwon/alexa_skills/search/pickles')


In [25]:
c = edatb.top_entity_dict("Donald Trump", most_c=5)
listing = [tupling[0] for tupling in c]
", ".join(listing)

'U.S., Trump, Russian, United States, Russia'

In [ ]:
import sys
sys.path.insert(0, 'C:/Users/User/Desktop/beaver/NumberWon/numberwon/alexa_skills/search')
print("done")
from entityDatabase import entityDatabase

In [1]:
import sys
sys.path.insert(0, 'C:/Users/User/Desktop/beaver/NumberWon/numberwon/alexa_skills/profiles/Profiles')
from Profile import Profile

In [3]:
p = Profile("Melanie")
p.pref_dict

{}

In [10]:
import numpy as np
npy = np.load('profiles_test_database.npy')
npy

array({'Megan': <profiles.Profiles.Profile.Profile object at 0x00000000060F2AC8>, 'Johnny': <profiles.Profiles.Profile.Profile object at 0x00000000060D7BE0>, 'Melanie': <profiles.Profiles.Profile.Profile object at 0x00000000060D7898>}, dtype=object)

In [23]:
d = UserDatabase('profiles_test_database.npy')
d.get_user_by_name('Megan').pref_dict['news']

['entertainment']

In [18]:
from UserDatabase import UserDatabase

In [24]:
from fanfiction import Fanfiction
from profile_skills import update_current_user

import sys
sys.path.insert(0, 'C:/Users/User/Desktop/beaver/NumberWon/numberwon/alexa_skills/profiles/Profiles')
from Profile import Profile
from UserDatabase import UserDatabase


File already exists:
	c:\users\user\desktop\beaver\dlibmodels\dlib_models\dlib_face_recognition_resnet_model_v1.dat
File already exists:
	c:\users\user\desktop\beaver\dlibmodels\dlib_models\shape_predictor_68_face_landmarks.dat


In [29]:
sys.path.insert(0, 'C:/Users/User/Desktop/beaver/NumberWon/numberwon/alexa_skills')
from profile_skills import *

yes_intent()

In [30]:
sys.path.insert(0, 'C:/Users/User/Desktop/beaver/NumberWon/numberwon/alexa_skills/search')
from entityDatabase import entityDatabase

In [ ]:
e = entityDatabase()
edatb = entityDatabase()
edatb.add_Folder_Database('C:/Users/User/Desktop/beaver/NumberWon/numberwon/alexa_skills/search/pickles')
c = edatb.top_entity_dict(EntityName, most_c=5)
listing = [tupling[0] for tupling in c]
print(", ".join(listing))